# Read data

In [ ]:
import pandas as pd
!./copy-from-pi.sh >> /dev/null
transfer_df = pd.io.parsers.read_csv('transfer-results.csv', parse_dates=["Timestamp"], index_col="Timestamp")
eth0_df = pd.io.parsers.read_csv('eth0-results.csv', parse_dates=["Timestamp"], index_col="Timestamp")
wlan1_df = pd.io.parsers.read_csv('wlan1-results.csv', parse_dates=["Timestamp"], index_col="Timestamp")
print("Number of records - transfer: {} eth0: {} wlan1: {}". \
    format(len(transfer_df.index), len(eth0_df.index), len(wlan1_df.index)))
print("Last two hours worth of transfer records...")
transfer_df[-12:] # show the last two hours of transfer records

# Last 24 hours

In [ ]:
import datetime
import numpy as np

rollingPeriod = 6

now = datetime.datetime.now()
dayago = now - datetime.timedelta(days=1)
merged_act = pd.merge(eth0_df[dayago : now], wlan1_df[dayago : now], left_index=True, right_index=True, how="outer", \
    suffixes=(("_eth0", "_wlan1")))
merged = pd.merge(transfer_df[dayago : now], merged_act[dayago : now], left_index=True, right_index=True, how="outer")
merged["Download"] = merged["Download"] / 1000 # change to kbits
merged["Upload"] = merged["Upload"] / 1000 # change to kbits
merged["NormPacketCount_eth0"] = merged["PacketCount_eth0"] / merged_act["Duration_eth0"] # packets / s
merged["NormPacketCount_wlan1"] = merged["PacketCount_wlan1"] / merged_act["Duration_wlan1"] # packets / s
merged = merged.drop(["Server ID", "Sponsor", "Server Name", "Distance", "Share", "IP Address", "PacketCount_eth0",
                      "Duration_eth0", "Interface_eth0", "PacketCount_wlan1", "Duration_wlan1", "Interface_wlan1"], 
                     axis=1)
merged = merged.resample('5min').mean() # align transfer_df, lan_df timestamps; shuffle data to round time intervals

merged["Upload"] = merged["Upload"].rolling(rollingPeriod).mean()
merged["Download"] = merged["Download"].rolling(rollingPeriod).mean()
merged["Ping"] = merged["Ping"].rolling(rollingPeriod).mean()
merged["NormPacketCount_eth0"] = (merged["NormPacketCount_eth0"] * 10).rolling(rollingPeriod).mean()
merged["NormPacketCount_wlan1"] = (merged["NormPacketCount_wlan1"] * 10).rolling(rollingPeriod).mean()
merged["NormPacketCount_comb"] = merged["NormPacketCount_eth0"] + merged["NormPacketCount_wlan1"] # *after* resample

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn
seaborn.set_style('dark')

fig = plt.figure(figsize=(9.5, 9))
ax1 = fig.add_subplot(3, 1, 1)
ax2 = fig.add_subplot(3, 1, 2)
ax3 = fig.add_subplot(3, 1, 3)
ax1.plot(merged["Upload"], color='g', linestyle='solid', label="Up (kbps)")
ax1.plot(merged["Download"],color='r', linestyle='solid', label="Down (kbps)")
ax1.set_title('Raw upload and download performance')
ax1.legend(loc='best')
ax2.plot(merged["Ping"], color='b', linestyle='solid', label="Ping (ms)")
ax2.plot([], color='k', linestyle='solid', label="LAN activity (packet count per 10s)")
ax2.set_title('Local LAN activity and ping to remote host')
ax2a = ax2.twinx()
ax2a.plot(merged["NormPacketCount_comb"], color='k', linestyle='solid', label="LAN activity (packet count per 10s)")
ax2.legend(loc='best')
ax3.plot(merged["Upload"] / merged["NormPacketCount_comb"], color='g', linestyle='solid', label="Up")
ax3.plot(merged["Download"] / merged["NormPacketCount_comb"], color='r', linestyle='solid', label="Down")
ax3.set_title('Upload and download, normalised by LAN activity')
ax3.legend(loc='best')